In [ ]:
!pip install pyagrum

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 53.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
from typing import List, Dict, Any, Tuple
import pyagrum as agr


In [ ]:
class Nodo:
    def __init__(self, nombre, dependencias, probabilidades, opciones):
        self.nombre = nombre
        self.dependencias = dependencias
        self.probabilidades = probabilidades
        self.opciones = opciones

class Grafo:
    def __init__(self):
        self.nodos = {}

    def añadirNodo(self, nodo):
        if not isinstance(nodo, Nodo):
            raise TypeError("El objeto debe ser de tipo Nodo")

        if nodo.nombre in self.nodos:
            raise ValueError("El nodo ya existe")

        self.nodos[nodo.nombre] = nodo
    def obtener_padres(self, nombre_nodo):
        """Devuelve la lista de nombres de los nodos padres."""
        if nombre_nodo not in self.nodos:
            raise ValueError("El nodo no está en el grafo")

        return self.nodos[nombre_nodo].dependencias

    def obtener_hijos(self, nombre_nodo):
        """Devuelve una lista de los nodos hijos."""
        hijos = []
        for nombre, nodo in self.nodos.items():
            if nombre_nodo in nodo.dependencias:
                hijos.append(nombre)
        return hijos

In [ ]:
def numDep(lector: Any):
    try:
        linea2 = next(lector)
        nDep = 0
        for i, valor in enumerate(linea2):
            try:
                float(valor.strip())
                nDep = i
                return nDep
            except ValueError:
                continue
        return len(linea2)
    except StopIteration:
        return 0


In [ ]:
def leerArchivo(nomArch: str):
    try:
        with open(nomArch, 'r', encoding='utf-8') as archivo:
            nombre = nomArch[:-4]
            lector = csv.reader(archivo, skipinitialspace=True)
            encabezados = next(lector)
            nDep = numDep(lector)
            dependencias = [h.strip() for h in encabezados[:nDep]]
            opciones = [h.strip() for h in encabezados[nDep:]]
            print(f"nDep inferido: {nDep}")
            print(f"Dependencias: {dependencias}")
            print(f"Opciones: {opciones}")
            archivo.seek(0) #Volver a la primera linea del archivo
            lector = csv.reader(archivo, skipinitialspace=True)
            next(lector) # Saltar el encabezado
            probabilidades: Dict[Tuple[str], Dict[str, float]] = {}
            for lineas in lector:
                if not lineas:
                    continue
                opcionesDep = tuple(v.strip() for v in lineas[:nDep])
                columnasProb = tuple(float(v.strip()) for v in lineas [nDep:])
                asignacionProb = dict(zip(opciones, columnasProb))
                probabilidades[opcionesDep] = asignacionProb
            nodo = Nodo(nombre, dependencias,probabilidades, opciones)
            return nodo
    except FileNotFoundError:
        print(f"Error: No se pudo abrir el archivo '{nomArch}'.")
        return None, None
    except Exception as e:
        print(f"Ocurrió un error inesperado: {e}")
        return None, None


def leer_dependencias(path_csv: str):
    df = pd.read_csv(path_csv)
    arcos = list(df[['Padre', 'Hijo']].itertuples(index=False, name=None))
    return arcos

def inferenciaEnumeracion(bn: agr.BayesNet, evidencia: Dict[str, str], consulta: str) -> Dict[str, float]:
    ie = agr.LazyPropagation(bn)
    ie.setEvidence(evidencia)
    resultado = ie.posterior(consulta)
    distribucion = {}
    for i in range(resultado.size()):
        etiqueta = bn.variable(consulta).label(i)
        probabilidad = resultado[i]
        distribucion[etiqueta] = probabilidad
    return distribucion

In [ ]:
import itertools

def prob_conjunta_dada_asignacion(grafo, asignacion):
    total = 1.0
    for nombre, nodo in grafo.nodos.items():
        valor = asignacion[nombre]
        if not nodo.dependencias:
            p = nodo.probabilidades.get((), {}).get(valor, 1)
        else:
            deps = tuple(asignacion[dep] for dep in nodo.dependencias)
            p = nodo.probabilidades.get(deps, {}).get(valor, 1)
        total *= p
    return total


def inferencia_por_enumeracion(grafo, consulta, evidencia):
    ocultas = [n for n in grafo.nodos if n not in evidencia and n != consulta]

    resultados = {}
    traza = []

    print(f" Variables ocultas: {ocultas}")
    print(f" Consulta: {consulta}")
    print(f" Evidencia: {evidencia}")

    # Recorremos cada valor posible de la variable de consulta
    for valor_consulta in grafo.nodos[consulta].opciones:
        total = 0.0
        dominios_ocultos = [grafo.nodos[var].opciones for var in ocultas]

        print(f"\n Evaluando P({consulta}={valor_consulta})")

        for combinacion in itertools.product(*dominios_ocultos):
            asignacion = evidencia.copy()
            asignacion[consulta] = valor_consulta

            # Añadir las combinaciones de las variables ocultas
            for var, val in zip(ocultas, combinacion):
                asignacion[var] = val

            p = prob_conjunta_dada_asignacion(grafo, asignacion)
            total += p
            traza.append(f"{asignacion} => P = {p:.6f}")
            print(f"   {asignacion} => P = {p:.6f}")

        resultados[valor_consulta] = total

    # Normalización
    alpha = 1.0 / sum(resultados.values())
    for k in resultados:
        resultados[k] *= alpha

    print("\n=== Distribución normalizada ===")
    for val, prob in resultados.items():
        print(f"P({consulta}={val} | evidencia) = {prob:.6f}")

    # Guardar traza en archivo
    with open("traza.txt", "w") as f:
        f.write("=== TRAZA DE INFERENCIA POR ENUMERACIÓN ===\n")
        for linea in traza:
            f.write(linea + "\n")
    return resultados



def main():
    grafo = Grafo()
    nodo_train = leerArchivo("Train.csv")
    nodo_rain = leerArchivo("Rain.csv")
    nodo_appt = leerArchivo("Appoinment.csv")
    nodo_maint = leerArchivo("Maintenance.csv")

    if not all([nodo_train, nodo_rain, nodo_appt, nodo_maint]):
        print(" Error: No se pudieron cargar todos los archivos.")
        return

    grafo.añadirNodo(nodo_train)
    grafo.añadirNodo(nodo_rain)
    grafo.añadirNodo(nodo_appt)
    grafo.añadirNodo(nodo_maint)

    print("\n ¡Red cargada exitosamente!")

    evidencia = {"Rain": "Light", "Maintenance": "No"}
    consulta = "Train"

    print(f"\n Ejecutando inferencia por enumeración para: {consulta}")
    resultado = inferencia_por_enumeracion(grafo, consulta, evidencia)



if __name__ == "__main__":
    main()


nDep inferido: 2
Dependencias: ['Rain', 'Maintenance']
Opciones: ['On time', 'Delay']
nDep inferido: 0
Dependencias: []
Opciones: ['None', 'Light', 'Heavy']
nDep inferido: 1
Dependencias: ['Train']
Opciones: ['Attend', 'Miss']
nDep inferido: 1
Dependencias: ['Rain']
Opciones: ['Yes', 'No']

 ¡Red cargada exitosamente!

 Ejecutando inferencia por enumeración para: Train
 Variables ocultas: ['Appoinment']
 Consulta: Train
 Evidencia: {'Rain': 'Light', 'Maintenance': 'No'}

 Evaluando P(Train=On time)
   {'Rain': 'Light', 'Maintenance': 'No', 'Train': 'On time', 'Appoinment': 'Attend'} => P = 0.100800
   {'Rain': 'Light', 'Maintenance': 'No', 'Train': 'On time', 'Appoinment': 'Miss'} => P = 0.011200

 Evaluando P(Train=Delay)
   {'Rain': 'Light', 'Maintenance': 'No', 'Train': 'Delay', 'Appoinment': 'Attend'} => P = 0.028800
   {'Rain': 'Light', 'Maintenance': 'No', 'Train': 'Delay', 'Appoinment': 'Miss'} => P = 0.019200

=== Distribución normalizada ===
P(Train=On time | evidencia) = 0.70

CASO DE PRUEBA: Ejemplo Gimnasio

Variable Consulta: Gains

In [ ]:
import itertools

def prob_conjunta_dada_asignacion(grafo, asignacion):
    total = 1.0
    for nombre, nodo in grafo.nodos.items():
        valor = asignacion[nombre]
        if not nodo.dependencias:
            p = nodo.probabilidades.get((), {}).get(valor, 1)
        else:
            deps = tuple(asignacion[dep] for dep in nodo.dependencias)
            p = nodo.probabilidades.get(deps, {}).get(valor, 1)
        total *= p
    return total


def inferencia_por_enumeracion(grafo, consulta, evidencia):
    ocultas = [n for n in grafo.nodos if n not in evidencia and n != consulta]

    resultados = {}
    traza = []

    print(f" Variables ocultas: {ocultas}")
    print(f" Consulta: {consulta}")
    print(f" Evidencia: {evidencia}")

    # Recorremos cada valor posible de la variable de consulta
    for valor_consulta in grafo.nodos[consulta].opciones:
        total = 0.0
        dominios_ocultos = [grafo.nodos[var].opciones for var in ocultas]

        print(f"\n Evaluando P({consulta}={valor_consulta})")

        for combinacion in itertools.product(*dominios_ocultos):
            asignacion = evidencia.copy()
            asignacion[consulta] = valor_consulta

            # Añadir las combinaciones de las variables ocultas
            for var, val in zip(ocultas, combinacion):
                asignacion[var] = val

            p = prob_conjunta_dada_asignacion(grafo, asignacion)
            total += p
            traza.append(f"{asignacion} => P = {p:.6f}")
            print(f"   {asignacion} => P = {p:.6f}")

        resultados[valor_consulta] = total

    # Normalización
    alpha = 1.0 / sum(resultados.values())
    for k in resultados:
        resultados[k] *= alpha

    print("\n=== Distribución normalizada ===")
    for val, prob in resultados.items():
        print(f"P({consulta}={val} | evidencia) = {prob:.6f}")

    # Guardar traza en archivo
    with open("traza.txt", "w") as f:
        f.write("=== TRAZA DE INFERENCIA POR ENUMERACIÓN ===\n")
        for linea in traza:
            f.write(linea + "\n")
    return resultados



def main():
    grafo = Grafo()
    nodo_suplementos = leerArchivo("Suplementos.csv")
    nodo_comida = leerArchivo("Comida.csv")
    nodo_entrenamiento = leerArchivo("Entrenamiento.csv")
    nodo_gains = leerArchivo("Gains.csv")
    nodo_trabajo = leerArchivo("Trabajo.csv")
    nodo_suenio = leerArchivo("Sueño.csv")

    if not all([nodo_suplementos, nodo_comida, nodo_entrenamiento, nodo_gains, nodo_trabajo, nodo_suenio]):
        print(" Error: No se pudieron cargar todos los archivos.")
        return

    grafo.añadirNodo(nodo_suplementos)
    grafo.añadirNodo(nodo_comida)
    grafo.añadirNodo(nodo_entrenamiento)
    grafo.añadirNodo(nodo_gains)
    grafo.añadirNodo(nodo_trabajo)
    grafo.añadirNodo(nodo_suenio)

    print("\n ¡Red cargada exitosamente!")

    evidencia = {"Suplementos": "Si", "Comida": "Si", "Sueño":"Malo", "Entrenamiento":"Suficiente"} ##aca se decide que se va a inferir
    consulta = "Gains"

    print(f"\n Ejecutando inferencia por enumeración para: {consulta}")
    resultado = inferencia_por_enumeracion(grafo, consulta, evidencia)



if __name__ == "__main__":
    main()


nDep inferido: 1
Dependencias: ['Trabajo']
Opciones: ['Si', 'No']
nDep inferido: 1
Dependencias: ['Trabajo']
Opciones: ['Si', 'No']
nDep inferido: 1
Dependencias: ['Trabajo']
Opciones: ['Suficiente', 'Insuficiente', 'Sobreentrenamiento']
nDep inferido: 4
Dependencias: ['Comida', 'Sueño', 'Entrenamiento', 'Suplementos']
Opciones: ['Si', 'No']
nDep inferido: 0
Dependencias: []
Opciones: ['Si', 'No']
nDep inferido: 2
Dependencias: ['Trabajo', 'Entrenamiento']
Opciones: ['Bueno', 'Medio', 'Malo']

 ¡Red cargada exitosamente!

 Ejecutando inferencia por enumeración para: Gains
 Variables ocultas: ['Trabajo']
 Consulta: Gains
 Evidencia: {'Suplementos': 'Si', 'Comida': 'Si', 'Sueño': 'Malo', 'Entrenamiento': 'Suficiente'}

 Evaluando P(Gains=Si)
   {'Suplementos': 'Si', 'Comida': 'Si', 'Sueño': 'Malo', 'Entrenamiento': 'Suficiente', 'Gains': 'Si', 'Trabajo': 'Si'} => P = 0.700000
   {'Suplementos': 'Si', 'Comida': 'Si', 'Sueño': 'Malo', 'Entrenamiento': 'Suficiente', 'Gains': 'Si', 'Trabajo'

Variable Consulta: Sueño

In [ ]:
import itertools

def prob_conjunta_dada_asignacion(grafo, asignacion):
    total = 1.0
    for nombre, nodo in grafo.nodos.items():
        valor = asignacion[nombre]
        if not nodo.dependencias:
            p = nodo.probabilidades.get((), {}).get(valor, 1)
        else:
            deps = tuple(asignacion[dep] for dep in nodo.dependencias)
            p = nodo.probabilidades.get(deps, {}).get(valor, 1)
        total *= p
    return total


def inferencia_por_enumeracion(grafo, consulta, evidencia):
    ocultas = [n for n in grafo.nodos if n not in evidencia and n != consulta]

    resultados = {}
    traza = []

    print(f" Variables ocultas: {ocultas}")
    print(f" Consulta: {consulta}")
    print(f" Evidencia: {evidencia}")

    # Recorremos cada valor posible de la variable de consulta
    for valor_consulta in grafo.nodos[consulta].opciones:
        total = 0.0
        dominios_ocultos = [grafo.nodos[var].opciones for var in ocultas]

        print(f"\n Evaluando P({consulta}={valor_consulta})")

        for combinacion in itertools.product(*dominios_ocultos):
            asignacion = evidencia.copy()
            asignacion[consulta] = valor_consulta

            # Añadir las combinaciones de las variables ocultas
            for var, val in zip(ocultas, combinacion):
                asignacion[var] = val

            p = prob_conjunta_dada_asignacion(grafo, asignacion)
            total += p
            traza.append(f"{asignacion} => P = {p:.6f}")
            print(f"   {asignacion} => P = {p:.6f}")

        resultados[valor_consulta] = total

    # Normalización
    alpha = 1.0 / sum(resultados.values())
    for k in resultados:
        resultados[k] *= alpha

    print("\n=== Distribución normalizada ===")
    for val, prob in resultados.items():
        print(f"P({consulta}={val} | evidencia) = {prob:.6f}")

    # Guardar traza en archivo
    with open("traza.txt", "w") as f:
        f.write("=== TRAZA DE INFERENCIA POR ENUMERACIÓN ===\n")
        for linea in traza:
            f.write(linea + "\n")
    return resultados



def main():
    grafo = Grafo()
    nodo_suplementos = leerArchivo("Suplementos.csv")
    nodo_comida = leerArchivo("Comida.csv")
    nodo_entrenamiento = leerArchivo("Entrenamiento.csv")
    nodo_gains = leerArchivo("Gains.csv")
    nodo_trabajo = leerArchivo("Trabajo.csv")
    nodo_suenio = leerArchivo("Sueño.csv")

    if not all([nodo_suplementos, nodo_comida, nodo_entrenamiento, nodo_gains, nodo_trabajo, nodo_suenio]):
        print(" Error: No se pudieron cargar todos los archivos.")
        return

    grafo.añadirNodo(nodo_suplementos)
    grafo.añadirNodo(nodo_comida)
    grafo.añadirNodo(nodo_entrenamiento)
    grafo.añadirNodo(nodo_gains)
    grafo.añadirNodo(nodo_trabajo)
    grafo.añadirNodo(nodo_suenio)

    print("\n ¡Red cargada exitosamente!")

    evidencia = {"Trabajo": "Si", "Entrenamiento": "Suficiente"} ##aca se decide que se va a inferir
    consulta = "Sueño"

    print(f"\n Ejecutando inferencia por enumeración para: {consulta}")
    resultado = inferencia_por_enumeracion(grafo, consulta, evidencia)



if __name__ == "__main__":
    main()

nDep inferido: 1
Dependencias: ['Trabajo']
Opciones: ['Si', 'No']
nDep inferido: 1
Dependencias: ['Trabajo']
Opciones: ['Si', 'No']
nDep inferido: 1
Dependencias: ['Trabajo']
Opciones: ['Suficiente', 'Insuficiente', 'Sobreentrenamiento']
nDep inferido: 4
Dependencias: ['Comida', 'Sueño', 'Entrenamiento', 'Suplementos']
Opciones: ['Si', 'No']
nDep inferido: 0
Dependencias: []
Opciones: ['Si', 'No']
nDep inferido: 2
Dependencias: ['Trabajo', 'Entrenamiento']
Opciones: ['Bueno', 'Medio', 'Malo']

 ¡Red cargada exitosamente!

 Ejecutando inferencia por enumeración para: Sueño
 Variables ocultas: ['Suplementos', 'Comida', 'Gains']
 Consulta: Sueño
 Evidencia: {'Trabajo': 'Si', 'Entrenamiento': 'Suficiente'}

 Evaluando P(Sueño=Bueno)
   {'Trabajo': 'Si', 'Entrenamiento': 'Suficiente', 'Sueño': 'Bueno', 'Suplementos': 'Si', 'Comida': 'Si', 'Gains': 'Si'} => P = 0.700000
   {'Trabajo': 'Si', 'Entrenamiento': 'Suficiente', 'Sueño': 'Bueno', 'Suplementos': 'Si', 'Comida': 'Si', 'Gains': 'No'} =

Variable Consulta: Suplementos

In [ ]:
import itertools

def prob_conjunta_dada_asignacion(grafo, asignacion):
    total = 1.0
    for nombre, nodo in grafo.nodos.items():
        valor = asignacion[nombre]
        if not nodo.dependencias:
            p = nodo.probabilidades.get((), {}).get(valor, 1)
        else:
            deps = tuple(asignacion[dep] for dep in nodo.dependencias)
            p = nodo.probabilidades.get(deps, {}).get(valor, 1)
        total *= p
    return total


def inferencia_por_enumeracion(grafo, consulta, evidencia):
    ocultas = [n for n in grafo.nodos if n not in evidencia and n != consulta]

    resultados = {}
    traza = []

    print(f" Variables ocultas: {ocultas}")
    print(f" Consulta: {consulta}")
    print(f" Evidencia: {evidencia}")

    # Recorremos cada valor posible de la variable de consulta
    for valor_consulta in grafo.nodos[consulta].opciones:
        total = 0.0
        dominios_ocultos = [grafo.nodos[var].opciones for var in ocultas]

        print(f"\n Evaluando P({consulta}={valor_consulta})")

        for combinacion in itertools.product(*dominios_ocultos):
            asignacion = evidencia.copy()
            asignacion[consulta] = valor_consulta

            # Añadir las combinaciones de las variables ocultas
            for var, val in zip(ocultas, combinacion):
                asignacion[var] = val

            p = prob_conjunta_dada_asignacion(grafo, asignacion)
            total += p
            traza.append(f"{asignacion} => P = {p:.6f}")
            print(f"   {asignacion} => P = {p:.6f}")

        resultados[valor_consulta] = total

    # Normalización
    alpha = 1.0 / sum(resultados.values())
    for k in resultados:
        resultados[k] *= alpha

    print("\n=== Distribución normalizada ===")
    for val, prob in resultados.items():
        print(f"P({consulta}={val} | evidencia) = {prob:.6f}")

    # Guardar traza en archivo
    with open("traza.txt", "w") as f:
        f.write("=== TRAZA DE INFERENCIA POR ENUMERACIÓN ===\n")
        for linea in traza:
            f.write(linea + "\n")
    return resultados



def main():
    grafo = Grafo()
    nodo_suplementos = leerArchivo("Suplementos.csv")
    nodo_comida = leerArchivo("Comida.csv")
    nodo_entrenamiento = leerArchivo("Entrenamiento.csv")
    nodo_gains = leerArchivo("Gains.csv")
    nodo_trabajo = leerArchivo("Trabajo.csv")
    nodo_suenio = leerArchivo("Sueño.csv")

    if not all([nodo_suplementos, nodo_comida, nodo_entrenamiento, nodo_gains, nodo_trabajo, nodo_suenio]):
        print(" Error: No se pudieron cargar todos los archivos.")
        return

    grafo.añadirNodo(nodo_suplementos)
    grafo.añadirNodo(nodo_comida)
    grafo.añadirNodo(nodo_entrenamiento)
    grafo.añadirNodo(nodo_gains)
    grafo.añadirNodo(nodo_trabajo)
    grafo.añadirNodo(nodo_suenio)

    print("\n ¡Red cargada exitosamente!")

    evidencia = {"Trabajo": "No"} ##aca se decide que se va a inferir
    consulta = "Suplementos"

    print(f"\n Ejecutando inferencia por enumeración para: {consulta}")
    resultado = inferencia_por_enumeracion(grafo, consulta, evidencia)



if __name__ == "__main__":
    main()

nDep inferido: 1
Dependencias: ['Trabajo']
Opciones: ['Si', 'No']
nDep inferido: 1
Dependencias: ['Trabajo']
Opciones: ['Si', 'No']
nDep inferido: 1
Dependencias: ['Trabajo']
Opciones: ['Suficiente', 'Insuficiente', 'Sobreentrenamiento']
nDep inferido: 4
Dependencias: ['Comida', 'Sueño', 'Entrenamiento', 'Suplementos']
Opciones: ['Si', 'No']
nDep inferido: 0
Dependencias: []
Opciones: ['Si', 'No']
nDep inferido: 2
Dependencias: ['Trabajo', 'Entrenamiento']
Opciones: ['Bueno', 'Medio', 'Malo']

 ¡Red cargada exitosamente!

 Ejecutando inferencia por enumeración para: Suplementos
 Variables ocultas: ['Comida', 'Entrenamiento', 'Gains', 'Sueño']
 Consulta: Suplementos
 Evidencia: {'Trabajo': 'No'}

 Evaluando P(Suplementos=Si)
   {'Trabajo': 'No', 'Suplementos': 'Si', 'Comida': 'Si', 'Entrenamiento': 'Suficiente', 'Gains': 'Si', 'Sueño': 'Bueno'} => P = 0.026010
   {'Trabajo': 'No', 'Suplementos': 'Si', 'Comida': 'Si', 'Entrenamiento': 'Suficiente', 'Gains': 'Si', 'Sueño': 'Medio'} => P =